In [1]:
import os, json, re
import random, time
import pandas as pd, numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
from dotenv import load_dotenv

from itertools import islice

# Util built for sharktank project
import sharktank_utils_mod as su
from sharktank_utils_mod import load_facts, PitchOrchestrator, metrics_calculation, PitchEditor, OrchestratorResponse, PitchEquipped

# Agno for llm agents
from agno.agent import Agent
from agno.models.groq import Groq
from agno.storage.agent.sqlite import SqliteAgentStorage
from agno.tools.duckduckgo import DuckDuckGoTools
# from agno.tools.wikipedia import WikipediaTools

# Other LLM utils
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

/opt/anaconda3/envs/sharktank/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[Knowledge Loader] ================================
>> Chunking Strategy: FIXED
>> Agent Storage File: tmp/pdf_rag.db
>> Agent Table Name: HBS_fixed_agent
>> ChromaDB Collection Name: HBS_fixed
>> ChromaDB Storage Path: ./chromadb_data
>> PDFs Being Processed:
   - https://raw.githubusercontent.com/sciencenerd880/LLM-DBT/main/data/pdfs/hbs_opportunities.pdf
   - https://raw.githubusercontent.com/sciencenerd880/LLM-DBT/main/data/pdfs/hbs_pitchdeck_sample.pdf



INFO Embedder not provided, using OpenAIEmbedder as default.

In [2]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
reference_models = su.REFERENCE_MODELS
agent_storage = su.agent_storage
edit_reference_models = su.EDIT_REFERENCE_MODELS

In [4]:
facts_store = load_facts()
pitches = {k:"" for k,_ in facts_store.items()}
metrics = {k:"" for k,_ in facts_store.items()}
time_stamps = {k:"" for k,_ in facts_store.items()}

In [9]:
# Main loop

orcheditor = PitchEquipped(reference=edit_reference_models, 
                          iterations=2,
                          orchestrator="deepseek-r1-distill-llama-70b"
                        )
goal = 'create a pitch with the given facts'

RUN_FIRST_N = True      # Change to False to run all
N = 1

cases_to_run = islice(facts_store.items(), N) if RUN_FIRST_N else facts_store.items()
for case, facts in tqdm(cases_to_run, desc="Generating pitches"):
    # case = 'facts_shark_tank_transcript_37_TIK PIK.txt'
    lean_facts = {
        'facts': facts['facts'],
        'product_description': facts['product_description']
    }
    try:
        pitch = orcheditor.orchestrate_with_edit(goal, lean_facts, verbose=True)
        pitches[case] = pitch
        metrics[case] = orcheditor.logs
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    except Exception as e:
        print(f"{case} error: {e}")
        pitches[case] = ""
        metrics[case] = []
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Reset for next run
    orcheditor.logs = []
    orcheditor.agents = {}

Generating pitches: 0it [00:00, ?it/s]

	>>> Subtasks: [{'name': 'Research and Outline Unique Value Proposition', 'description': "Use DuckDuckGo to gather information on similar products and market trends to highlight GarmaGuard's unique features such as natural ingredients and portability.", 'assigned_tools': ['duckduckgo']}, {'name': 'Calculate Key Business Metrics', 'description': 'Use the calculator to determine metrics like average order value, customer lifetime value, and profit projections based on the provided sales and cost data.', 'assigned_tools': ['calculator']}, {'name': 'Craft Compelling Business Case', 'description': "Combine research findings and calculated metrics to create a persuasive pitch outlining GarmaGuard's market success and growth potential.", 'assigned_tools': ['duckduckgo', 'calculator']}]


INFO Dividing 476000.0 by 14000.0 to get 34.0

INFO Multiplying 12.99 and 0.066 to get 0.8573400000000001

INFO Multiplying 100000.0 and 0.33 to get 33000.0

...

ERROR    Error calling Groq API: Error code: 400 - {'error': {'message': 'response_format` json_object cannot be   
         combined with tool/function calling', 'type': 'invalid_request_error'}}

WARNING  Attempt 1/1 failed: {"error":{"message":"response_format` json_object cannot be combined with             
         tool/function calling","type":"invalid_request_error"}}

ERROR    Failed after 1 attempts. Last error using Groq(deepseek-r1-distill-llama-70b)

[02:24<?] 0/2 | Running Pitch Editing Iterations:   0%|           ?it/s
Generating pitches: 1it [02:24, 144.50s/it]

{"error":{"message":"response_format` json_object cannot be combined with tool/function calling","type":"invalid_request_error"}}
 for case: {'name': 'GarmaGuard', 'type': 'Natural garment and fabric cleanser', 'functionality': 'Uses natural propellants to eliminate odor, freshen fabric, and control dirt and grime. It kills 99% of germs and odor-causing bacteria.', 'application': 'Spray directly onto clothes.', 'unique_selling_point': 'First of its kind, designed for on-the-go use to keep clothes fresh and clean without the need for frequent washing or dry cleaning.'}


In [6]:
pitches.keys()

dict_keys(['facts_shark_tank_transcript_0_GarmaGuard.txt', 'facts_shark_tank_transcript_0_Roadie.txt', 'facts_shark_tank_transcript_0_TouchUp Cup.txt', 'facts_shark_tank_transcript_10_Deux.txt', 'facts_shark_tank_transcript_10_Fish Fixe.txt', 'facts_shark_tank_transcript_10_Hidrent.txt', 'facts_shark_tank_transcript_11_Dude Wipes.txt', 'facts_shark_tank_transcript_11_Love & Pebble.txt', 'facts_shark_tank_transcript_11_Pink Picasso.txt', 'facts_shark_tank_transcript_11_Sheets Laundry Club.txt', 'facts_shark_tank_transcript_11_Zach & Zoe Sweet Bee Farm.txt', 'facts_shark_tank_transcript_12_Elf Grams.txt', 'facts_shark_tank_transcript_12_Ornament Anchor.txt', "facts_shark_tank_transcript_12_Santa's Enchanted Mailbox.txt", "facts_shark_tank_transcript_12_Wendy's Gnome Shop.txt", 'facts_shark_tank_transcript_13_Banana Loca.txt', 'facts_shark_tank_transcript_13_Liberate.txt', 'facts_shark_tank_transcript_13_MAXPRO SmartConnect.txt', 'facts_shark_tank_transcript_13_Tenikle.txt', 'facts_shark_

In [7]:
print(pitches["facts_shark_tank_transcript_0_GarmaGuard.txt"])

In [8]:
print(pitches["facts_shark_tank_transcript_0_Roadie.txt"])